# Zpracování MARC souborů pomocí knihovny Pymarc

Tento notebook vás provede procesem zpracování MARC souborů za použití knihovny Pymarc. MARC (Machine-Readable Cataloging) je standardní formát pro záznamy knihovních katalogů a metadat. Ukážeme si, jak načíst marcové soubory, jak procházet jejich záznamy, jak extrahovat potřebná data a jak je uložit do CSV a Excel souborů. <br>

Tento notebook je určený jak pro začátečníky, tak pro ty, kteří se chtějí seznámit s knihovnou Pymarc a zpracováním MARC dat v Pythonu.

## Předpoklady

Notebook nepředpokládá hluboké znalosti Pythonu, ale základní znalost programování bude užitečná.<br> 

## Struktura notebooku

Tento notebook je rozdělen do několika částí:

0. **Příprava**: Přidáme potřebné knihovny, které budeme používat ke zpracování marcového souboru. 

1. **Načtení MARC souboru**: Ukážeme si, jak načíst marcový soubor a vypsat jednotlivé záznamy.

2. **Získání dat**: Naučíme se, jak získat specifická data z marcového záznamu, jako jsou názvy, autoři a žánry.

4. **Export do CSV a Excelu**: V závěrečné části si ukážeme, jak uložit získaná data do CSV a Excel souborů pro další analýzu.


## Další zdroje

- [LearnPython.org](https://www.learnpython.org/): Tento online kurz nabízí výuku jazyka Python pro začátečníky i pokročilé. Může být užitečným zdrojem pro ty, kteří chtějí rozšířit své znalosti Pythonu.

- [W3Schools.com/Python](https://www.w3schools.com/python/): Obsáhlý tutoriál, který provází i některými oblíbenými knihovnamy Pythonu. 



### 0. Příprava 
Jako první si musíme nainstalovat knihovny, se kterými budeme pracovat. Knihovny jsou balíčky funkcí, které nejsou součástí základu jazyka Python. <br>
Knihovny nainstalujeme pomocí příkazu `%pip install <jmeno_knihovny>` . Pak je do našeho notebooku přidáme pomocí příkazu `import <jmeno_knihovny> (as alias)`. K funkcím knihovny se pak přistupuje `jmeno_knihovny.jmeno_funkce` <br> 
Pokud z knihovny chceme využít pouze jednu funkci, přidáme ji pomocí `from <jmeno_knihovny> import <jmeno_funkce>`

In [ ]:
# Install libraries
%pip install pandas 
%pip install pymarc 
%pip install openpyxl

# Add libraries 
import pandas as pd
from pymarc import MARCReader


### 1. Načtení MARC souboru

Naše data jsou uložena v knihovním formátu MARC s koncovnkou .mrc. Pro práci budeme používat funkci `MARCReader` z knihovnu `pymarc`, která data ze souboru načte a záznamy rozdělí, takže k nim pak můžeme přistoupit jednotlivě. <br>  

Marcové soubory jsou v adresáři <b>data/marc</b>. Jednotlivé báze jsou pak uloženy pod názvem <b>ucla_\<kod_baze\>.mrc</b>.<br>
Výsledná cesta i se souborem je  <b>data/marc/ucla_\<kod_baze\>.mrc</b> <br>

K dispozici jsou následující báze:

* <b>ret</b> - Retrospektivní bibliografie české literatury 1770–1945

* <b>smz</b> - Bibliografie českého literárního samizdatu

* <b>int</b> - Bibliografie českého literárního internetu

* <b>cle</b> - Bibliografie českého literárního exilu (1948–1989)

#### 1.1 Výpis záznamu

Vypíšeme si, jak takový MARC záznam vypadá. <br>
Nejprve si vybereme bázi a určíme k ní cestu. Příkaz `format()` nám do cesty přidá kód naší vybrané báze z proměnné `base`<br>
Pak si určíme, kolikátý záznam chceme vypsat. To definujeme pomocí proměnné `ith`.<br> 
Pak otevřeme marcový soubor a postupně ho projdeme. Funkce `enumerate()` nám záznamy navíc očísluje od 0. Díky tomu budeme moct vypsat i-tý záznam.<br>
Abychom nemuseli procházet celý soubor, napíšeme slovo `break`, které ukončí procházení souboru.

In [ ]:
# Select base
base = 'cle' 

# Path to marc file
database = 'data/marc/ucla_{base}.mrc'.format(base = base)

# ith record to print
ith = 5

# Open file
with open(database, 'rb') as data:
    # Read file
    reader = MARCReader(data)

    # Iterate through records in marc file 
    for i,record in enumerate(reader):        
        
        # If i is our record 
        if i == ith:
            # Print record
            print(record)

            # Terminate the loop
            break


Vidíme, že marcový soubor má jasnou strukuru. Má několik polí, která jsou označena zpravidla třemi číslicem, případně třemi písmeny. Každý kód má svou vnitřní logiku, např. pole pro věcné popisy vždy začínají číslicí ``6XX``. <br>
Za číslem - tagem pole se obvyke nachází dva indikátory. Pokud indikátor není definovaný, píše se místo něj zpětné lomítko (\\). <br>
Většina polí se dělí na podpole. Nachází se za dolarem (``$``) a označují se buď jedním písmenem, nebo číslicí.  



<div class='alert alert-block alert-info'>
    <b>Try It!</b>  Pomocí parametru ith nastavíme kolikátý záznam chceme vypsat (indexování začíná od 0). Pokud bychom chtěli vypsat všechny záznamy do i-tého záznamu, změníme `if i == ith:` na `if i <= ith:` a smažeme příkaz `break`.
</div>



#### 1.2 Výpis jednotlivých polí

Pro práci s databází pravděpodobně nebudeme potřebovat všechna pole, proto si teď ukážeme, přistupovat k jednotlivým polím v záznamu. <br>
Vypíšeme si jen číslo záznamu, název, autora a žánr. K některým polím se můžeme dostat přes tečkovou notaci (`record._`), k ostatním musíme přistupovat přes funkci `get_fields(<cislo pole>)`, která vrátí všechna pole s číslem v závorce.  

In [ ]:
ith = 5

# Open file
with open(database, 'rb') as data:
    
    # Read marc file
    reader = MARCReader(data)

    # Iterate through records in marc file 
    for i, record in enumerate(reader):

        # If i is our record 
        if i == ith:
            # Print marc file
            # Some fields are accessible via dot notation, e.g. record.leader or record.title
            print("Record: " + record.leader)
        
            # It is better to check if a a field exists (= is not None)
            # Printing a None value triggers an error
            if record.title is not None:
                print("Title: " + record.title)
            if record.author is not None:
                print("Author: " + record.author)
            
            # We call a function .get_fields() if a field is not accessible via dot notation   
            if record.get_fields('655') is not None:     
                # Almost all field are accessible via square brackets  
                print("Genre: " + record['655']['a'])
            break        

#### 1.3 Počet záznamů

Také by nás mohlo zajímat, kolik záznamů je v dané databázi. K tomu si vytvoříme samostatnou funkci `number_of_records(database)`, kterou pak jen zavoláme. Vlastní funkce definujeme pomocí slova `def`<br> 
Funkce jako vstup bere cestu k marcové databázi. Uvnitř funkce si otevře, projde a při každém záznamu si připočítá jedničku k počítadlu `counter`.  

In [ ]:
# To define out own functions we use 'def'
def number_of_records(database):
    
    with open(database, 'rb') as data:
        # Reac marc file
        reader = MARCReader(data)
        # Create a counter 
        counter = 0
        # Underscore ignores value that we don't need
        for _ in reader:
            counter += 1
    
    # Function returns value         
    return counter 

print("There are " + str(number_of_records(database)) + " records in the database.")        

### 2. Získání dat 

S marcovým dokumentem se nepracuje příliš dobře, proto je lepší si data uložit do jednodušší tabulky. V této fázi si musíme ujasnit, jaká data budeme chtít. V našem příkladu budeme chtít uložit název, autora, autorův kód, rok vydání a pak pole '``600 $a``','``650 $a``', '``655 $a``' a '``773 $t``'. <br>

#### 2.1 Marcová pole

Veškeré záznamy začínající číslem 6XX jsou věcné údaje o záznamu. Tato pole se mohou opakovat. <br>     
Pod polem '600' se skrývají osoby, o kterých záznam je nebo případně osoby, kterým je záznam dedikován. <br> 
Pod polem '650' se nacházejí věcné termíny/téma, tzn. o čem záznam je. <br>
Pod polem '655' pak najdeme žánr daného záznamu. Na rozdíl od polí '600' a '650' by pole '655' mělo být přítomné u každého záznamu. <br>
Záznamy začínající čísly 76X - 78X se nazývají propojovací pole a slouží pro zápis zdrojového (773) nebo recenzovaného (787) dokumentu. <br>



In [ ]:
# ith record to print
ith = 10

# Open file 
with open(database, 'rb') as data:
    # Read marc file
    reader = MARCReader(data)

    # Iterate through records in marc file 
    for i,record in enumerate(reader):        
        
        # If i is our record 
        if i == ith:
            
            print("Record: " + record.leader)
            
            # If field exists we print it
            if record.get_fields('600') is not None:   
                # There may be more fields under the tag, so we iterate through all of them   
                for field in record.get_fields('600'): 
                    print("Personal name: " + field['a'])
                
                
            # If field exists we print it   
            if record.get_fields('650') is not None:    
                # There may be more fields under the tag, so we iterate through all of them  
                for field in record.get_fields('650'): 
                    print("Topical term: " + field['a'])
            
            # If field exists we print it
            if record.get_fields('655') is not None: 
                # There may be more fields under the tag, so we iterate through all of them     
                for field in record.get_fields('655'): 
                    print("Genre/Form: " + field['a'])

            # Terminate the loop
            break


#### 2.2 Selekce polí

Pro ukládání máme připravenou funkci `save_to_dict(record, dictionary, field_list)`, která nám jeden záznam (`record`) uloží do dictionary (struktury v Pythonu) `marc_dictionary`. <br>
Struktura dictionary je tvořena dvojicí klíč - hodnota. K hodnotě se dostaneme skrz klíč v hranatých závorkách. `dict[<klic>] = <hodnota>`<br> 
Protože není strategické si do tabulky ukládat všechna pole, podpole a indikátory, funkci `save_to_dict` kromě záznamu předáme také list polí `field_list`, která chceme uložit. <br>

Jelikož se některá pole (např. 700) mohou opakovat, je dobré si hodnoty za každé pole uložit do listu (kolekce v jazyce Python). List je seznam hodnot, např. textu - stringů, čísel - integerů nebo float atd. <br>
Kvůli obecnosti si do listu uložíme veškeré hodnoty. Je jednodušší, když pracujeme pouze s jedním typem (např. list nebo string), než když jsou některé hodnoty v listu a některé jsou pouze string. Pokud jsme si jisti, že v původních záznamech se pole nemůže opakovat, můžeme hodnoty uložit samostatně mimo list. <br>    

In [ ]:
def save_to_dict(record, marc_dictionary, field_list):
    if not record is None:
        try:
            # Iterate through list 'field_list'
            for field_tags in field_list:
                # Key name in dictionary
                dict_key_name =  field_tags[0]

                # Field tag
                tag =  field_tags[1]

                # Subfield tag
                subfield_tag =  field_tags[2]
                
                # List for adding values to dictionary 
                dict_add_list = []
                
                # Iterate through all fields with tag 'tag'
                for field in record.get_fields(tag):
                    
                    # If field doesn't have any subfields, add whole field to 'dict_add_list'
                    if subfield_tag is None:
                        dict_add_list.append(field.data)  
                    
                    # If  subfield tag is slice instance (we only want a part of a field that does not have a subfield)
                    # add the slice to 'dict_add_list'
                    elif isinstance(subfield_tag, slice):
                        dict_add_list.append(field.data[subfield_tag])     

                    # If the field contains our subfield tag, add the subfield to 'dict_add_list'
                    elif '$'+subfield_tag in str(field):  
                        dict_add_list.append(str(field[subfield_tag]))

                # We need to use dot notation for accessing leader
                if tag == 'LDR':
                    dict_add_list.append(record.leader)        

                # Add 'dict_add_list' to 'dict_key_name'         
                marc_dictionary[dict_key_name].append(dict_add_list)
        except Exception as error:
            print("Exception: " + type(error).__name__)  
            print("LDR: " + str(record.leader))   
    return marc_dictionary 

print("Function saved.")

Naši funkci teď využijeme k tomu, abychom si data z marcového souboru vytáhli. Nejprve si do listu `field_list` napíšeme, jaké hodnoty chceme. <br>
`field_list` sestává z tuplů (kolekce v jazyce Python, do češtiny přeložené jako n-tice), kde každý tuple vypadá následovně. Na první pozici je název klíče, pod kterým se rozhodneme pole uložit, na druhé pozici tag pole a na třetí tag podpole, např. ('author', '100', 'a') nebo ('author', '100', None). <br>
Pak si vytvoříme proměnnou  `marc_dictionary`, do kterého budeme postupně hodnoty přidávat pomocí naší funkce `save_to_dict`. Klíče `marc_dictionary` jsou první hodnoty z tuplů ve `field_list`, hodnoty pak data z marcového záznamu v jednom listu.<br>
Nakonec data převedeme do datové struktury DataFrame (která je podobná např. excelovské tabulce), se kterou je mnohem jednodušší pracovat. <br>
Řádky v DataFramu reprezentují jednotlivé záznamy, sloupce pak jeden typ (např. jméno autora).<br>

In [ ]:
with open(database, 'rb') as data:
    reader = MARCReader(data)
    # List of values we want to save
    field_list = [('title', '245', 'a'),
                ('author', '100', 'a'),
                ('author code', '100', '7'),
                # Date of publication is in the 8th to 11th place, so we use slice function 
                ('year', '008', slice(7,11, None)), # Indexing starts at 0. 
                ('figures', '600', 'a'),
                ('description', '650', 'a'),
                ('genre', '655', 'a'),
                ('magazine', '773', 't')]
    
    # Dictionary for saving our data
    marc_dictionary = {}
    
    # Iterate through tuples in 'field_list'
    for t in field_list:
        
        # Key name is first in the tuple     
        dict_key_name = t[0]
        
        # We add the key to the dictionary and an empty list (that we will later fill) as a value
        marc_dictionary[dict_key_name] = []
    
    # Iterate through all records in the database  
    for record in reader:
        
        # Call our function save_to_dict
        marc_dictionary = save_to_dict(record, marc_dictionary, field_list)

# Create a DataFramu from 'marc_dictionary'       
df = pd.DataFrame.from_dict(marc_dictionary)

print("Marc file saved to DataFrame df.")

<div class='alert alert-block alert-info'>
    <b>Try It!</b>  Další hodnoty ze záznamů získáme jednoduše. Jen přidáme další tuple do seznamu za poslední hodnotu, v našem případě za ('magazine', '773', 't'). <br>
    Můžeme přidat i pole, která nemají podpole. Např. pokud bychom chtěli přidat pole 005, které obsahuje informace o poslední změně záznamu, přidáme ('latest transaction','005', None). <br>
    Záznam o leaderu přidáme takto - ('leader', 'LDR', None).  
</div>


In [ ]:
# Print last 5 records in the DataFrame 'df'
df.tail()

#### 2.3 Úprava dat 

Vidíme, že jména za sebou mají přebytečnou čárku, kterou jednoduše odstraníme. Stejně tak názvy přebytečné lomítko. Díky funkcím `apply()` a `lambda` můžeme změnit všechny hodnoty ve sloupci pomocí jednoho řádku kódu. Ve funkci `lambda` si definujeme, jak data chceme upravit. Funkcí `apply()` použijeme funkci `lambda` na všechny hodnoty. <br> 
Všechna data jsou uložena v listu, proto musíme projít všechny hodnoty v list zvlášť. <br>  
Funkce vezme jednotlivé listy ve sloupci a zjistí, zda list není prázdný ( tedy se zeptá, zda je velikost listu větší než 0 `len(y) > 0`). Pokud není, funkce projde hodnoty v listu a uloží si je bez koncové čárky. To udělá pomocí funkce `strip(' ,')`, která případně odstraní koncové čárky nebo mezery u jmen. U názvů použijeme funkci `strip(' /')`, která nás zbaví přebytečného lomítka. Pokud je prázdný, funkce list nijak nezmění.   

In [ ]:
# Save name and surname without comma at the end of the string
df['figures'] = df['figures'].apply(lambda x: [y.strip(' ,') if len(y) > 0  else y for y in x])  
df['author'] = df['author'].apply(lambda x: [y.strip(' ,') if len(y) > 0 else y for y in x]) 

# Save title without slash at the end of the string  
df['title'] = df['title'].apply(lambda x: [y.strip(' /') if len(y) > 0 else y for y in x])  

# Print last 5 records in the DataFrame 'df'
df.tail()

### 3. Export do CSV a excelu

V následujícím kroku si budeme chtít data exportovat do formátu CSV. Jelikož CSV tabulka nepracuje dobře s listy, hodnoty v listu spojíme středníkem do jednoho stringu pomocí funkce `join()`. K tomu opět využijeme lambda funkci. <br>

###### Funkce `join()` může spojit i jednotlivá písmena ve stringu. Abychom předešli tomu, že mezi každým písmenkem budeme mít středník, v lambda funkci pomocí `isinstance()` otestujeme, zda jsou data opravdu v listu. 


In [ ]:
# Iterate through column in DataFrame 'df'
for column in df.columns:
    
    # Join all values in the list with a semicolon ';' 
    df[column] = df[column].apply(lambda x: ';'.join(x) if isinstance(x, list) else x )

df.tail()

Nakonec DataFrame uložíme do CSV a Excelu do adresáře 'data/csv', resp. 'data/excel'.  

In [ ]:
out_csv = 'data/csv/out_{base}.csv'.format(base = base)

# Save DataFrame to CSV format
df.to_csv(out_csv, encoding = 'utf8', sep = ",", index=False)   

print("Data saved to csv.")

out_excel = 'data/excel/out_{base}.xlsx'.format(base = base)

# Save DataFrame to Excel format
df.to_excel(out_excel,  index=False) 

print("Data saved to xlsx.")